<a href="https://colab.research.google.com/github/Jefffish09/MachineLearning/blob/dev/NER/bert_ner_simpletransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

* https://github.com/ThilinaRajapakse/simpletransformers
* https://simpletransformers.ai/

Data Source:

* https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus


In [1]:
!pip install -U -q seqeval
!pip install -U simpletransformers

     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 215kB 17.1MB/s 
     |████████████████████████████████| 1.9MB 42.7MB/s 
     |████████████████████████████████| 7.5MB 44.6MB/s 
     |████████████████████████████████| 2.0MB 55.7MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 1.2MB 47.4MB/s 
     |████████████████████████████████| 3.2MB 47.8MB/s 
     |████████████████████████████████| 317kB 57.8MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 163kB 55.6MB/s 
     |████████████████████████████████| 112kB 51.4MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 4.6MB 46.6MB/s 
     |████████████████████████████████| 133kB 60.2MB/s 
     |████████████████████████████████| 102kB 15.1MB/s 
     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 122kB 55.9MB/s

In [1]:
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel
from seqeval.metrics import classification_report

In [2]:
seed = 2021

In [3]:
# Get the raw dataset

!curl -s -LO https://github.com/MahmooudTaha/NLP-2019/raw/master/ner_dataset.csv.zip
!unzip -o ner_dataset.csv.zip
!rm -f ner_dataset.csv.zip


class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

df = pd.read_csv("ner_dataset.csv", encoding="Windows-1252").fillna(method="ffill")

getter = SentenceGetter(df)

sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
labels = [[s[2] for s in sentence] for sentence in getter.sentences]
tag_values = list(set(df["Tag"].values))

Archive:  ner_dataset.csv.zip
  inflating: ner_dataset.csv         


In [4]:
tr_sents, test_sents, tr_labels, test_labels = train_test_split(sentences, labels, random_state=seed, test_size=0.1)

train_data = []
for i, pair in enumerate(zip(tr_sents, tr_labels)):
  sent, label = pair
  for ii in zip(sent, label):
    l = list(ii)
    l.insert(0, i)
    train_data.append(l)
train_data = pd.DataFrame(
    train_data, columns=["sentence_id", "words", "labels"]
)

test_data = []
for i, pair in enumerate(zip(test_sents, test_labels)):
  sent, label = pair
  for ii in zip(sent, label):
    l = list(ii)
    l.insert(0, i)
    test_data.append(l)
test_data = pd.DataFrame(
    test_data, columns=["sentence_id", "words", "labels"]
)

In [5]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Configure the model
# https://simpletransformers.ai/docs/usage/
model_args = {
    "output_dir": "outputs/",
    "best_model_dir": "outputs/best_model",
    "num_train_epochs": 100,
    "train_batch_size": 128,
    "learning_rate": 4e-5,
    "max_seq_length": 128,
    "max_grad_norm": 1.0,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "optimizer": "AdamW",
    "labels_list": tag_values,
    "do_lower_case": True,
    "evaluate_during_training": True,
    "evaluate_during_training_verbose": True,
    "evaluate_during_training_steps": 1000,
    "eval_batch_size": 128,
    "warmup_steps": 0,
    "warmup_ratio": 0.06,
    "weight_decay": 0,
    "use_early_stopping": True,
    "early_stopping_patience": 3,
    "early_stopping_metric": "eval_loss",
    "early_stopping_delta": 0.001,
    "early_stopping_consider_epochs": True,
    "early_stopping_metric_minimize": True,
    "logging_steps": 10,
    "reprocess_input_data": True,
    "no_save": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "overwrite_output_dir": True,
    "use_multiprocessing": True,
    "use_multiprocessing_for_evaluation": False,
    "no_cache": True
}

model = NERModel(
    # https://simpletransformers.ai/docs/ner-specifics/
    # https://huggingface.co/transformers/pretrained_models.html
    "bert",
    "bert-base-uncased",
    args=model_args,
    use_cuda=True
    )

# Train the model
model.train_model(
    train_data,
    eval_data=test_data,
    show_running_loss=True
    )

# Evaluate the model
result, model_outputs, preds_list = model.eval_model(
    test_data,
    # acc=classification_report
    )

test_data_new = test_data.groupby("sentence_id")["labels"].apply(list).reset_index(name="grouped_labels")
true_list = test_data_new["grouped_labels"].tolist()
report = classification_report(y_true=true_list, y_pred=preds_list, digits=2)
print(report)

INFO:filelock:Lock 140660220264720 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

INFO:filelock:Lock 140660220264720 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock
INFO:filelock:Lock 140660076169168 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

INFO:filelock:Lock 140660076169168 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be

...

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.11404411769226978, 'precision': 0.8261711870376893, 'recall': 0.8364835948644793, 'f1_score': 0.8312954102427786}
INFO:simpletransformers.ner.ner_model: Patience of 3 steps reached
INFO:simpletransformers.ner.ner_model: Training terminated.
INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/4796 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.11404411769226978, 'precision': 0.8261711870376893, 'recall': 0.8364835948644793, 'f1_score': 0.8312954102427786}


              precision    recall  f1-score   support

         art       0.38      0.27      0.31        49
         eve       0.33      0.33      0.33        30
         geo       0.84      0.90      0.87      3712
         gpe       0.96      0.92      0.94      1649
         nat       0.36      0.53      0.43        17
         org       0.70      0.66      0.68      2033
         per       0.79      0.80      0.79      1699
         tim       0.87      0.88      0.87      2027

   micro avg       0.83      0.84      0.83     11216
   macro avg       0.65      0.66      0.65     11216
weighted avg       0.83      0.84      0.83     11216

